In [ ]:
!pip install selenium

In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import csv 
import urllib.request
from time import sleep
from time import time
from random import randint


browser = webdriver.Chrome(executable_path= 'C:\ProgramData\Anaconda3\Lib\site-packages\jupyterlab\chromedriver.exe')

In [ ]:
# define some data structure for further use
result_list=[]

In [ ]:
# define your own function (a block of code for reuse) "start_request"
def start_request(q):
    r = browser.get("https://en.tripadvisor.com.hk/"+q)
    print("crlawling "+q)
    return r

In [ ]:
def parse(text):
    
   sleep(randint(0,3))   # Add some time to wait to load the page
   container1 = browser.find_elements_by_xpath('//*[@id="taplc_hotel_review_atf_hotel_info_web_component_0"]')
   nbrcontainer = len(container1)
   mydict = {}
   pm = []

   for i in range(0, nbrcontainer):
        container1 = browser.find_elements_by_xpath('//*[@id="taplc_hotel_review_atf_hotel_info_web_component_0"]')
        container3 = browser.find_elements_by_xpath("//div[@class='eZQpm _T Z BB']")
        container4 = browser.find_elements_by_xpath("//*[@id='ABOUT_TAB']/div[2]")
        container5 = browser.find_elements_by_xpath("//div[@class='dKfjB']")
        container2 = browser.find_elements_by_xpath('//*[@id="taplc_hotel_review_atf_hotel_info_web_component_0"]')
        results = container1[i]
        result1 = container3[i]
        result4 = container4[i]
        result5 = container5[i]
        result2 = container2[i]
        try:
            N = results.find_element_by_xpath('//*[@id="HEADING"]')
            mydict['name'] = N.text
            
            try: 
                try:
                    reviews = results.find_element_by_xpath('//span[@class="HFUqL"]')
                    mydict['No_of_reviews'] = reviews.text          
                    rate = result1.find_element_by_xpath("//span[@class='bvcwU P']")
                    mydict['Rates'] = rate.text
                    Hotelclass = result4.find_element_by_xpath("//span[contains(@class,'fyiTP S2')]//*[name()='svg']").get_attribute("aria-label")
                    mydict['Hotel_class'] = Hotelclass
        
                except Exception as e:
                        break 
                
                PR = result5.find_element_by_xpath("//div[@class='cJdpk Ci']")
                PriceRange = PR.text[:-45].split("-", 1)
                mydict['MinPrice'] = PriceRange[0][3:8]
                mydict['MaxPrice'] = PriceRange[1][4:10]

            except:
                mydict['MaxPrice'] = "0"
                mydict['MinPrice'] = "0"
                
            Test = result5.find_element_by_xpath("//*[contains(text(), 'NUMBER OF ROOMS')]")
            Number_of_Rooms_Boolean = Test.is_displayed()
            if Number_of_Rooms_Boolean == True:
                location = result5.find_element_by_xpath('(//div[@class="cJdpk Ci"])[last()-1]')
                mydict['Location'] = location.text
            else: 
                location = result5.find_element_by_xpath('(//div[@class="cJdpk Ci"])[last()]')
                mydict['Location'] = location.text

            result_list.append(mydict)
        except Exception as e:
            print(e)
            print('not____________________________found')
            mydict['name'] = 'null'
            
            result_list.append(mydict)


In [ ]:
with open('Top_300.csv') as f:    
    reader = csv.DictReader(f)    
    for row in reader:         
        req = row['Hotel URL']  
        text = start_request(req)
        parse(text)
        sleep(randint(1,3))

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(result_list)
df.to_csv('Detailed Hotelinfo_v2.csv')
df